In [ ]:
import tempfile
import os

PROJECT_ID = "gcp-mira-develop"
REGION = "us-central1"
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["OPENAI_API_KEY"] = ""

import sys
sys.path.append("../../common/src")
sys.path.append("../src")

from google.cloud import storage

from services.query.query_service import datasource_from_url
from services.query.data_source import DataSource

storage_client = storage.Client(project=PROJECT_ID)

# get datasource class for doc_url
doc_url = "gs://"
data_source = datasource_from_url(doc_url, None, storage_client)

docs_processed = []
with tempfile.TemporaryDirectory() as temp_dir:
  data_source_files = data_source.download_documents(doc_url, temp_dir)

  # counter for unique index ids
  index_base = 0

  for data_source_file in data_source_files:
    doc_name = data_source_file.doc_name
    index_doc_url = data_source_file.src_url
    doc_filepath = data_source_file.local_path

    print(f"processing [{doc_name}]")

    # text_chunks = data_source.chunk_document(doc_name,
    #                                           index_doc_url,
    #                                           doc_filepath)
    # print (text_chunks)

    slide_chunks = data_source.chunk_pdf_slides(doc_name, index_doc_url, doc_filepath)
    for slide in slide_chunks:
      print(slide["text_chunks"])
      print(slide["image_b64"][:700])
    
    # cleanup temp local file
    os.remove(doc_filepath)